In [1]:
!pip install beir

     |████████████████████████████████| 52 kB 694 kB/s 
     |████████████████████████████████| 78 kB 4.3 MB/s 
     |████████████████████████████████| 8.4 MB 34.3 MB/s 
     |████████████████████████████████| 379 kB 29.2 MB/s 
     |████████████████████████████████| 4.9 MB 34.0 MB/s 
     |████████████████████████████████| 3.1 MB 17.4 MB/s 
     |████████████████████████████████| 3.3 MB 30.1 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
  Created wheel for beir: filename=beir-0.2.3-py3-none-any.whl size=45805 sha256=f1e1866227be602fc6b5b5e48386dd343fd21bdf773e5551c9beee6c48c6a398
  Stored in directory: /root/.cache/pip/wheels/af/29/4c/261c842753d34921b8b35c1b8679720114f8536654767430dd
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=265179 sha256=77d2725df0

In [2]:
! pip install datasets transformers

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
     |████████████████████████████████| 132 kB 44.8 MB/s 
     |████████████████████████████████| 243 kB 39.1 MB/s 
     |████████████████████████████████| 160 kB 42.6 MB/s 
     |████████████████████████████████| 271 kB 45.8 MB/s 
     |████████████████████████████████| 192 kB 46.4 MB/s 


# Partie 1

In [3]:
import pandas as pd

In [4]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
dataset = "dbpedia"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

  0%|          | 0/4635922 [00:00<?, ?it/s]

In [6]:
from datasets import load_dataset, load_metric

datasets = load_dataset("squad_v2")

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
all_good_entries = []

for i in qrels:
  # Check if query is a question (because we want a question-answer thing)
  if "QALD2" in i:
    # get question
    question = queries[i]
    # for every entry that correspond to a question
    for j in qrels[i]:
      # get context
      context = corpus[j]['text']
      # check if it's long enough for us
      context_tokens = context.split(' ')
      context_words = [word.lower() for word in context_tokens]
      if len(context_words) >= 50 and context not in all_good_entries:
        # big enough, add it in the list of good context
        all_good_entries.append(context)

In [8]:
dico = {}
for i in range(len(datasets['validation'])):
  context = datasets  ['validation'][i]['context']
  if context not in dico:
    new_dico = {}
    new_dico['title'] = datasets['validation'][i]['title']
    new_dico['question'] = datasets['validation'][i]['question']
    new_dico['answers'] = datasets['validation'][i]['answers']
    dico[context] = new_dico

In [9]:
datasets['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [10]:
all_questions = []
all_contexts = []
all_titles = []
all_answers = []

for context in dico.keys():
  all_contexts.append(context)
  all_questions.append(dico[context]['question'])
  all_titles.append(dico[context]['title'])
  all_answers.append(dico[context]['answers'])

In [11]:
all_contexts = all_contexts + all_good_entries

In [12]:
len(all_contexts)

11852

In [13]:
df = pd.DataFrame({'question': pd.Series(all_questions), 'context': pd.Series(all_contexts), 'title': pd.Series(all_titles), 'answers': pd.Series(all_answers)})
df

,question,context,title,answers
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,Normans,"{'text': ['France', 'France', 'France', 'Franc..."
1,Who was the duke in the battle of Hastings?,"The Norman dynasty had a major political, cult...",Normans,"{'text': ['William the Conqueror', 'William th..."
2,What is the original meaning of the word Norman?,"The English name ""Normans"" comes from the Fren...",Normans,"{'text': ['Viking', 'Norseman, Viking', 'Norse..."
3,When was the Duchy of Normandy founded?,"In the course of the 10th century, the initial...",Normans,"{'text': ['911', '911', '911'], 'answer_start'..."
4,Who upon arriving gave the original viking set...,"Before Rollo's arrival, its populations did no...",Normans,"{'text': ['Rollo', 'Rollo', 'Rollo'], 'answer_..."
...,...,...,...,...
11847,NaN,The World Meteorological Organization (WMO) is...,NaN,NaN
11848,NaN,The World Veterans Federation (WVF) is the wor...,NaN,NaN
11849,NaN,World Vision International is an Evangelical C...,NaN,NaN
11850,NaN,ZF Electronics GmbH (formerly known as Cherry ...,NaN,NaN


In [14]:
seen = set()
seen_add = seen.add
unique_questions = [x for x in all_questions if not (x in seen or seen_add(x))]
q_a = {}

for question in unique_questions:
  a = df.loc[df['question'] == question]
  l = list(a.index)
  q_a[question] = l

In [15]:
len(unique_questions)

1204

In [16]:
unique_questions[:5]

['In what country is Normandy located?',
 'Who was the duke in the battle of Hastings?',
 'What is the original meaning of the word Norman?',
 'When was the Duchy of Normandy founded?',
 'Who upon arriving gave the original viking settlers a common identity?']

# Partie 2

In [17]:
from sentence_transformers import SentenceTransformer
import sentence_transformers.util

In [18]:
print(len(unique_questions))
print(len(df.context.to_list()))

1204
11852


In [19]:
def index_model_cosine_similarity(model, questions, contexts):
  embeddings = model.encode(contexts, show_progress_bar=True)
  question_embeddings = model.encode(questions, show_progress_bar=True)
  similarity = sentence_transformers.util.pytorch_cos_sim(question_embeddings, embeddings)
  return similarity

def index_model_dot_product_similarity(model, questions, contexts):
  embeddings = model.encode(contexts, show_progress_bar=True)
  question_embeddings = model.encode(questions, show_progress_bar=True)
  similarity = sentence_transformers.util.dot_score(question_embeddings, embeddings)
  return similarity

def get_ranks(res):
  all_ranks = []

  for i in range(len(res)):
    similarities = res[i]
    a = sorted(range(len(similarities)), key=lambda k: similarities[k])
    a.reverse()
    question = unique_questions[i]
    for rank in range(len(a)):
      if a[rank] in q_a[question]:
        all_ranks.append(rank + 1)
        break
  return all_ranks

In [21]:
def inverse(n):
  return 1/n

In [22]:
%%time
model_distilbert_v4 = SentenceTransformer('msmarco-distilbert-base-v4')

similarity_distilbert_v4 = index_model_cosine_similarity(model_distilbert_v4, unique_questions, df.context.to_list())
rank_distilbert_v4 = get_ranks(similarity_distilbert_v4)
result_distilbert_v4 = map(inverse, rank_distilbert_v4)
print(sum(list(result_distilbert_v4)) / len(rank_distilbert_v4))

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.7372709088377434
CPU times: user 12min 59s, sys: 3.13 s, total: 13min 2s
Wall time: 13min 8s


In [23]:
%%time
model_distilbert_v3 = SentenceTransformer('msmarco-distilbert-base-v3')

similarity_distilbert_v3 = index_model_cosine_similarity(model_distilbert_v3, unique_questions, df.context.to_list())
rank_distilbert_v3 = get_ranks(similarity_distilbert_v3)
result_distilbert_v3 = map(inverse, rank_distilbert_v3)
print(sum(list(result_distilbert_v3)) / len(rank_distilbert_v3))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.7473299182677691
CPU times: user 13min 6s, sys: 5.11 s, total: 13min 11s
Wall time: 13min 13s


In [24]:
%%time
model_roberta_v3 = SentenceTransformer('msmarco-roberta-base-v3')

similarity_roberta_v3 = index_model_cosine_similarity(model_roberta_v3, unique_questions, df.context.to_list())
rank_roberta_v3 = get_ranks(similarity_roberta_v3)
result_roberta_v3 = map(inverse, rank_roberta_v3)
print(sum(list(result_roberta_v3)) / len(rank_roberta_v3))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.6636146515360238
CPU times: user 15min 10s, sys: 7.35 s, total: 15min 18s
Wall time: 15min 19s


In [25]:
%%time
model_distilbert_tas = SentenceTransformer('msmarco-distilbert-base-tas-b')

similarity_distilbert_tas = index_model_dot_product_similarity(model_distilbert_tas, unique_questions, df.context.to_list())
rank_distilbert_tas = get_ranks(similarity_distilbert_tas)
result_distilbert_tas = map(inverse, rank_distilbert_tas)
print(sum(list(result_distilbert_tas)) / len(rank_distilbert_tas))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.8203144880079584
CPU times: user 12min 59s, sys: 4.82 s, total: 13min 4s
Wall time: 13min 6s


In [26]:
%%time
model_roberta_firstp = SentenceTransformer('msmarco-roberta-base-ance-firstp')

similarity_roberta_firstp = index_model_dot_product_similarity(model_roberta_firstp, unique_questions, df.context.to_list())
rank_roberta_firstp = get_ranks(similarity_roberta_firstp)
result_roberta_firstp = map(inverse, rank_roberta_firstp)
print(sum(list(result_roberta_firstp)) / len(rank_roberta_firstp))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.14k [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.7139793512717401
CPU times: user 15min 17s, sys: 7.07 s, total: 15min 24s
Wall time: 15min 27s


# Partie 3

In [29]:
import faiss
import numpy as np

In [30]:


embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id.values)

# # Retrieve the 10 nearest neighbours
# D, I = index.search(np.array([embeddings[5415]]), k=10)

NameError: ignored